In [ ]:
!pip install av==10.0.0
!pip install numpy==1.24.3
!pip install Pillow==9.5.0
!pip install streamlit==1.28.0
!pip install streamlit-webrtc==0.47.1
!pip install torch==2.0.0
!pip install ultralytics==8.0.201
!pip install opencv-python-headless==4.8.0.74
!pip install requests==2.28.2


  Using cached streamlit_webrtc-0.47.1-py3-none-any.whl (874 kB)
  Using cached aiortc-1.7.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.9 MB)
  Using cached aioice-0.9.0-py3-none-any.whl (24 kB)
  Using cached pyee-11.1.0-py3-none-any.whl (15 kB)
  Using cached pylibsrtp-0.10.0-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Using cached dnspython-2.6.1-py3-none-any.whl (307 kB)
  Using cached ifaddr-0.2.0-py3-none-any.whl (12 kB)
  Using cached torch-2.0.0-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
!pip install --upgrade requests


In [ ]:
pip install ultralytics


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
%cd "/content/drive/My Drive/RDD2022_all_countries"


/content/drive/My Drive/RDD2022_all_countries


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import collections
import cv2
import bs4 as bs
import math
import glob
from xml.etree import ElementTree
import xml.etree.ElementTree as ET
from tqdm.notebook import tqdm
import tarfile
from bs4 import BeautifulSoup
from xml.dom import minidom
import os
from pathlib import Path
import glob
from tqdm import tqdm
import random
import shutil
import os

%matplotlib inline

In [ ]:
import zipfile
import os

# Chemin vers le fichier RDD2022.zip dans Google Drive
zip_path = '/content/drive/MyDrive/RDD2022.zip'

# Répertoire de destination pour l'extraction du fichier RDD2022.zip
extract_path = '/content/tmp'

# Extraire le contenu du fichier RDD2022.zip dans un répertoire temporaire
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Liste des fichiers ZIP extraits
extracted_files = os.listdir(extract_path)

# Extraire chaque fichier ZIP individuel
for file in extracted_files:
    if file.endswith('.zip'):
        # Chemin complet du fichier ZIP à extraire
        full_path = os.path.join(extract_path, file)
        # Répertoire de destination pour l'extraction du fichier ZIP
        destination_path = '/content/extracted/' + file.replace('.zip', '')
        # Extraire le fichier ZIP
        with zipfile.ZipFile(full_path, 'r') as zip_ref:
            zip_ref.extractall(destination_path)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/RDD2022.zip'

In [ ]:
def convertPascal2YOLOv8(filePath):

    class_mapping = {
        "D00": 0,
        "D10": 1,
        "D20": 2,
        "D40": 3,
        "D01": 4,
        "D11": 5,
        "D43": 6,
        "D44": 7,
        "D50": 8
    }

    # reading content
    file = open(filePath, "r")
    contents = file.read()

    # parsing
    soup = bs.BeautifulSoup(contents, 'xml')
    image_size = soup.find_all("size")[0]
    image_width = int(image_size.find_all("width")[0].get_text())
    image_height = int(image_size.find_all("height")[0].get_text())
    # print("w,h :", image_width, image_height)

    # Process Bounding Box
    objects = soup.find_all("object")

    # Placeholder
    bounding_box_list = []
    class_list = []

    for object in objects:

        # Object Class
        _class = object.find_all("name")[0].get_text()

        # Map the class to int number, if not defined > 10
        _class = class_mapping.get(_class, 10)
        class_list.append(_class)

        # Object Bounding Box
        _xmin = float(object.find_all("xmin")[0].get_text())
        _ymin = float(object.find_all("ymin")[0].get_text())
        _xmax = float(object.find_all("xmax")[0].get_text())
        _ymax = float(object.find_all("ymax")[0].get_text())

        # Convert to YOLOv8 Annotation
        # class x_center y_center width height
        w = (_xmax - _xmin)
        h = (_ymax - _ymin)
        cx = _xmin + (w/2)
        cy = _ymin + (h/2)

        # Normalize
        w = round((w / image_width), 4)
        h = round((h / image_height), 4)
        cx = round((cx / image_width), 4)
        cy = round((cy / image_height), 4)

        _bbox = [cx, cy, w, h]

        # print(_class, cx, cy, w, h)

        bounding_box_list.append(_bbox)

    # Get the filename
    outputFilename = os.path.split(filePath)[1]
    outputFilename = outputFilename.replace(".xml", ".txt")

    # Output Path
    outputDir = Path(filePath).parents[2]
    outputDir = outputDir / "labels"

    # Check if the directory exists
    if not os.path.exists(outputDir):
        os.makedirs(outputDir)

    # Final output path
    outputPath = outputDir / outputFilename
    # print(outputPath)

    # Write to .txt file
    with open(outputPath, 'w') as f:
        for i in range(len(class_list)):

            # Filter the class, drop unused class
            # 0: D00 > Longitudinal Crack
            # 1: D10 > Transverse Crack
            # 2: D20 > Alligator Crack
            # 3: D40 > Potholes

            if class_list[i] < 4:
                anno = str(class_list[i]) + " " +  str(bounding_box_list[i][0]) + " " +  str(bounding_box_list[i][1]) + " " +  str(bounding_box_list[i][2]) + " " +  str(bounding_box_list[i][3]) + "\n"
                f.write(anno)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Dataset Root Directory
ROOTDIR = "/content/drive/My Drive/RDD2022_all_countries"

# Liste des répertoires de pays
CountryListDir = ["/Czech/train/images/Czech/train/annotations/xmls",
                  "/China_MotorBike/train/annotations/xmls",
                  "/China_Drone/train/annotations/xmls",
                  "/United_States/train/annotations/xmls" ,
                  "/India/train/annotations/xmls",
                  "/Japan/train/annotations/xmls",

                    ]



for CountryDir in CountryListDir:

    CountryDir = ROOTDIR + CountryDir
    fileList = sorted(glob.glob(CountryDir + "/*.xml"))

    # Processing all the annotation
    for file in tqdm(fileList):
        convertPascal2YOLOv8(file)
        # break

100%|██████████| 10526/10526 [40:49<00:00,  4.30it/s]


In [ ]:
def CopyDatasetSplit(baseDir):

    # Split the training data to train and validation data due to lack of annotation on test data
    # Seed
    random.seed(1337)

    # Output Directory
    # !!! Change this to your clone folder
    baseOutputDir = "/content/drive/My Drive/RDD2022_all_countries/"
    countryName = Path(baseDir).parents[0]
    countryName = os.path.split(countryName)[1]

    baseImageDir = baseDir + "images/"
    baseAnnotDir = baseDir + "labels/"

    image_list_all = sorted(glob.glob(baseImageDir + "/*"))
    annot_list_all = sorted(glob.glob(baseAnnotDir + "/*"))

    # Drop any images that doesn't have annotation (background)
    # Or just leave it at some percentage of the dataset
    backgroundImages_Percentage = 0.1
    image_list = []
    annot_list = []

    dataset_length_all = len(image_list_all)
    max_background_image = int(dataset_length_all * backgroundImages_Percentage)
    _counter = 0

    for i in range(len(annot_list_all)):

        with open(annot_list_all[i]) as f:
            _annot = f.read()

            # Annotation not empty
            if _annot:
                # Check if the index is within the valid bounds of image_list_all
                if i < len(image_list_all):
                    image_list.append(image_list_all[i])
                    annot_list.append(annot_list_all[i])
            elif _counter < max_background_image:
                # Check if the index is within the valid bounds of image_list_all
                if i < len(image_list_all):
                    image_list.append(image_list_all[i])
                    annot_list.append(annot_list_all[i])
                    _counter += 1

    # Dataset length
    dataset_length = len(image_list)
    # print(dataset_length, len(annot_list))

    split_ratio = 0.9
    middle_point = round(split_ratio * dataset_length)

    # Create random list number using seed
    numberList = list(range(0, dataset_length))
    random.shuffle(numberList)
    trainNumberList = numberList[:middle_point]
    validNumberList = numberList[middle_point:]
    print("Training/Validation Samples :", len(trainNumberList), len(validNumberList))

    # Training images and labels
    print("Copying training images and labels for", countryName)
    for i in tqdm(trainNumberList):

        # Images
        outputImagesDir = baseOutputDir + countryName + "/images/train/"
        if not os.path.exists(outputImagesDir):
            os.makedirs(outputImagesDir)

        shutil.copy2(image_list[i], outputImagesDir)

        # Annotations
        outputAnnotDir = baseOutputDir + countryName + "/labels/train/"
        if not os.path.exists(outputAnnotDir):
            os.makedirs(outputAnnotDir)

        shutil.copy2(annot_list[i], outputAnnotDir)
        # print(outputImagesDir, outputAnnotDir)

    # Validation images and labels
    print("Copying validation images and labels for", countryName)
    for i in tqdm(validNumberList):

        # Images
        outputImagesDir = baseOutputDir + countryName + "/images/val/"
        if not os.path.exists(outputImagesDir):
            os.makedirs(outputImagesDir)

        shutil.copy2(image_list[i], outputImagesDir)

        # Annotations
        outputAnnotDir = baseOutputDir + countryName + "/labels/val/"
        if not os.path.exists(outputAnnotDir):
            os.makedirs(outputAnnotDir)

        shutil.copy2(annot_list[i], outputAnnotDir)
        # print(outputImagesDir, outputAnnotDir)

# baseDir = "../dataset/RDD2022/RDD2022_all_countries/Japan/train/"
# CopyDatasetSplit(baseDir)


In [ ]:
# Base Directory
ROOTDIR = "/content/drive/My Drive/RDD2022_all_countries/"

# Use only japan india
CountryListDir = ["/Japan/train/",
                  "/India/train/",
                #   "RDD2022_all_countries/China_Drone/train/",
                #   "RDD2022_all_countries/China_MotorBike/train/",
                #   "RDD2022/RDD2022_all_countries/Czech/train/",
                #   "RDD2022/RDD2022_all_countries/Norway/Norway/train/",
                #   "RDD2022/RDD2022_all_countries/United_States/United_States/train/",
]

for CountryDir in CountryListDir:
    CountryDir = ROOTDIR + CountryDir
    CopyDatasetSplit(CountryDir)

Training/Validation Samples : 3291 366
Copying training images and labels for Japan


100%|██████████| 3291/3291 [16:29<00:00,  3.33it/s]


Copying validation images and labels for Japan


100%|██████████| 366/366 [01:50<00:00,  3.30it/s]


Training/Validation Samples : 1894 210
Copying training images and labels for India


100%|██████████| 1894/1894 [09:41<00:00,  3.26it/s]


Copying validation images and labels for India


100%|██████████| 210/210 [01:05<00:00,  3.19it/s]


In [ ]:
!tree ./ -d


In [ ]:
# Importer les bibliothèques nécessaires
from yolov5 import YOLO

# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Chemins d'accès aux données d'entraînement et de validation pour l'Inde
INDIA_TRAIN_DIR = "/content/drive/My Drive/INDIA/images/train"
INDIA_VAL_DIR = "/content/drive/My Drive/INDIA/images/val"

# Chemin d'accès au fichier YAML contenant les informations sur vos données
_data = "/chemin/vers/votre/fichier.yaml"  # Assurez-vous de remplacer par votre propre chemin

# Project Name
_project = "runs/RDD_JapanIndia"
_name = "Baseline_YOLOv8Small_Filtered"

# Epoch et warmup epoch
_epoch = 100
_warmup_epochs = int(_epoch * 0.05)

# Créer une instance du modèle YOLOv8
model = YOLO('yolov8s.pt')

# Entraîner le modèle en utilisant les données d'entraînement et de validation pour l'Inde
results = model.train(data={"train": INDIA_TRAIN_DIR, "val": INDIA_VAL_DIR},  # Utiliser les chemins d'accès directs ici
                      epochs=_epoch,
                      warmup_epochs=_warmup_epochs,
                      batch=32,
                      imgsz=640,
                      save_period=10,
                      workers=1,
                      project=_project,
                      name=_name,
                      seed=1337,
                      cos_lr=True,
                      mosaic=0.0,
                     )


In [ ]:
https://github.com/oracl4/RoadDamageDetection/tree/main

In [ ]:

# Chemin racine des données
ROOTDIR = "/content/drive/My Drive/RDD2022_all_countries"

# Dictionnaire des chemins d'accès pour chaque ensemble de données (train et val)
data_paths = {
    "path": ROOTDIR,
    "train": [
        #"Japan/images/train",
        "India/images/train",
        #"Czech/images/train",
        #"United_States/images/train",
        #"China_Drone/images/train",
        #"China_MotorBike/images/train"
    ],
    "val": [
        #"Japan/images/val",
        "India/images/val",
        #"Czech/images/val",
        #"United_States/images/val",
        #"China_Drone/images/val",
        #"China_MotorBike/images/val"
    ]
}

# Dictionnaire des noms de classe
class_names = {
    0: "Longitudinal Crack",
    1: "Transverse Crack",
    2: "Alligator Crack",
    3: "Potholes"
}

# Fusionner les deux dictionnaires
data = {**data_paths, "names": class_names}

# Enregistrer le dictionnaire dans un fichier YAML
with open("data2_all_countries.yaml", "w") as yaml_file:
    yaml.safe_dump(data, yaml_file)


NameError: name 'yaml' is not defined

In [ ]:
from ultralytics import YOLO

In [ ]:

# Chemins d'accès aux données d'entraînement et de validation pour l'Inde
data_all_countries = "/content/drive/My Drive/data_INDIA_all_countries.yaml"

# Project Name
_project = "runs/RDD_JapanIndia"
_name = "Baseline_YOLOv8Small_Filtered"

# Epoch et warmup epoch
_epoch = 10
_warmup_epochs = int(_epoch * 0.05)

# Créer une instance du modèle YOLOv8
model = YOLO('models/yolov8s.pt')


# Entraîner le modèle en utilisant les données d'entraînement et de validation pour l'Inde
results = model.train(data=data_all_countries,  # Utiliser les chemins d'accès directs ici
                      epochs=_epoch,
                      warmup_epochs=_warmup_epochs,
                      batch=32,
                      imgsz=640,
                      save_period=10,
                      workers=1,
                      project=_project,
                      name=_name,
                      seed=1337,
                      cos_lr=True,
                      mosaic=0.0,
                     )


100%|██████████| 21.5M/21.5M [00:00<00:00, 210MB/s]


Ultralytics YOLOv8.1.19 🚀 Python-3.10.12 torch-2.1.0+cu121 CPU (AMD EPYC 7B12)
engine/trainer: task=detect, mode=train, model=models/yolov8s.pt, data=/content/drive/My Drive/data_INDIA_all_countries.yaml, epochs=10, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=10, cache=False, device=None, workers=1, project=runs/RDD_JapanIndia, name=Baseline_YOLOv8Small_Filtered, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=1337, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_c

RuntimeError: Dataset '/content/drive/My Drive/data_INDIA_all_countries.yaml' error ❌ 
Dataset '/content/drive/My Drive/data_INDIA_all_countries.yaml' images not found ⚠️, missing path '/content/datasets/C:/Users/MSI/Documents/RDD2022_all_countries/India/images/val'
Note dataset download directory is '/content/datasets'. You can update this in '/root/.config/Ultralytics/settings.yaml'